# Plots of passband ripple transform

Refer to: 

https://tomverbeure.github.io/2020/10/11/Designing-Generic-FIR-Filters-with-pyFDA-and-Numpy.html

https://tomroelandts.com/articles/how-to-create-a-configurable-filter-using-a-kaiser-window




In [ ]:
import torchsig.transforms.functional as F
import torchsig.utils.dsp as dsp
from torchsig.utils.dsp import (
    torchsig_real_data_type,
    torchsig_complex_data_type
)

import numpy as np
import scipy.signal as signal
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
fs = 1.0          # Sampling rate
cutoff = 0.2       # Cutoff frequency (Hz)
rp = 6.0              # Passband ripple (dB)
order = 5          # Filter order
numtaps = 127       # FIR filter length

# Design prototype Type 1 filter
b, a = signal.cheby1(order, rp, cutoff, fs=fs, btype='low')

# Generate impulse response of filter
t, h = signal.dimpulse((b, a, 1/fs), n=numtaps)
fir_coeffs = h[0].squeeze()

# Calculate frequency response
w, h0 = signal.freqz(fir_coeffs, fs=fs, worN=8000)

# Create subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# Magnitude plot
#mag = 20 * np.log10(np.abs(h0))
mag = (np.abs(h0))
ax1.plot(w, mag)
ax1.set_ylabel('Magnitude (dB)')
ax1.grid(True)
#ax1.set_xlim(0, fs/2 if fs > 1 else np.pi)

# Phase plot (unwrapped)
phase = np.unwrap(np.angle(h0))
ax2.plot(w, phase)
ax2.set_ylabel('Phase (radians)')
ax2.set_xlabel('Frequency (Hz)' if fs > 1 else 'Normalized Frequency')
ax2.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
def passband_ripple(
    data: np.ndarray,
    filter_coeffs: np.ndarray,
    normalize: bool = False
) -> np.ndarray:
    """Functional for passband ripple transforms.

    Args:
        data (np.ndarray): Complex valued IQ data samples.
        filter_coeffs (np.ndarray): FIR filter coeffecients with desired ripple characteristics.
        normalize (bool): Normalize filter coefficients for energy preservation. Default False.

    Returns:
        np.ndarray: Filtered data.

    """
    N = len(data)    

    if normalize: # filter energy normalization
        energy = np.sum(np.abs(filter_coeffs)**2)
        filter_coeffs = filter_coeffs / np.sqrt(energy)
    
    data = np.convolve(data, filter_coeffs)
    #data = data[-N:] # retain data size
    return data.astype(torchsig_complex_data_type)

In [ ]:
# test response with white noise data
rng = np.random.default_rng(42)

# parameters
N = 10000    # data size
ripple = 4.2 # passband ripple (mag dB)
cutoff = 0.12 # cutoff frequency (norm to Fs=1.0)
order = 10    # filter order
numtaps = 255 # FIR filter length

# impulsive noise input
data_in = dsp.noise_generator(
    N       = N,
    power   = 1.0, 
    color   = 'white',
    continuous = False,
    rng     = rng 
)

# design filter
fs = 1.0          # Sampling rate
b, a = signal.cheby1(order, ripple, cutoff, fs=fs, btype='low')
t, h = signal.dimpulse((b, a, 1/fs), n=numtaps)
fir_coeffs = h[0].squeeze()

input_power = np.sum(np.abs(data_in)**2)/len(data_in)
data_out = passband_ripple(
    data = data_in,
    filter_coeffs = fir_coeffs,
    normalize = True
)
output_power = np.sum(np.abs(data_out)**2)/len(data_out)

### Plots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# Calculate frequency response
D = np.fft.fftshift(np.fft.fft(data_out, norm="ortho"))
M = len(D)
freqs = np.fft.fftshift(np.fft.fftfreq(M))

# Full Magnitude plot
mag = np.abs(D)
ax1.plot(freqs, 20*np.log10(mag))
ax1.set_ylim(-100)
ax1.set_ylabel('Magnitude (dB)')
ax1.grid(True)

# Focus Magnitude plot
locut = int(round((0.5-cutoff)*M))
hicut = int(round((0.5+cutoff)*M))
ax2.plot(freqs[locut:hicut], mag[locut:hicut])
ax2.set_ylabel('Magnitude (lin)')
ax2.grid(True)

### Estimate ripple
peak_inds, _ = signal.find_peaks(mag, height=0.1, distance=M/20)
peak_vals = mag[peak_inds]

trough_inds, _ = signal.find_peaks(-mag, height=-10.0, distance=M/20)
trough_vals = mag[trough_inds]

print('input_power: ', input_power)
print('output_power: ', output_power)
print('peak values: ', peak_vals[peak_vals > 0.5])
print('trough values: ', trough_vals[trough_vals > 0.5])

ripple_est = np.mean(peak_vals[peak_vals > 0.5]) - np.mean(trough_vals[trough_vals > 0.5])
print('ripple_est: ', ripple_est)
print('ripple_est_db: ', 20*np.log10(1 + ripple_est))

#### 